In [1]:
import os 
from glob import glob
import pdal 
import laspy
import numpy as np 
import time 

/home/ljp238/miniconda3/envs/samlidar/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
from uvars import eba_laz_dir, eba_tif_dir,eba_lazclf_dir,transect_names

In [3]:
def pdal_quick_info(laz_path):
    reader = pdal.Reader(laz_path) # sensitive to resolution, can pass argument here 
    pipeline = reader.pipeline()
    qi = pipeline.quickinfo 
    return qi,pipeline

def pdal_pc_classes(pipeline):
    pipeline.execute()
    arrays = pipeline.arrays
    if len(arrays) == 0:
        print("No point data found in the file.")
    
    classification_values = arrays[0]['Classification']
    unique_classes = np.unique(classification_values)
    print(f"Unique classifications: {unique_classes}")
    return unique_classes

def generate_dtm(laz, tif, res=10,outfn="mean"):
    ti = time.perf_counter()

    tif = tif.replace('.tif', f"_DTM{str(res)}_{outfn}.tif")

    if os.path.isfile(tif):
        print(f"file alredy exist...\n{tif}")
        return tif 
    """Extracts ground points and generates a DTM."""
    print(f"Generating DTM: {laz} \n-> {tif}")

    pipeline = pdal.Reader(laz)
    pipeline |= pdal.Filter.expression(expression="Classification == 2")
    pipeline |= pdal.Writer.gdal(
        filename=tif, gdalopts="tiled=yes, compress=deflate", nodata=-9999,
        output_type=outfn, resolution=res
    )
    
    pipeline.execute()
    tf = time.perf_counter() - ti 
    print(f"DTM saved to {tif}")
    print(f"run.time = {tf/60} min(s)")
    return tif

def clf_pointcloud(laz, lazo, method="smrf"):
    ti = time.perf_counter()
    print("""Classifies ground points in a point cloud and saves to LAZ.""")
    lazo = lazo.replace('.laz', f'_{method}.laz')

    if os.path.isfile(lazo):
        print(f"file aready created {lazo}")
        return lazo 
    
    pipeline = pdal.Reader(laz)
    pipeline |= pdal.Filter.expression(expression="Classification != 7")
    pipeline |= pdal.Filter.assign(assignment="Classification[:]=0")
    if method == 'smrf':
        pipeline |= pdal.Filter.smrf()
    elif method == 'csf':
        pipeline |= pdal.Filter.csf()
    
    pipeline |= pdal.Writer.las(filename=lazo, compression="laszip")
    #https://pdal.io/en/latest/stages/writers.las.html
    pipeline.execute()
    print(f"Reclassified point cloud saved to \n{lazo}")
    tf = time.perf_counter() - ti 
    print(f"run.time = {tf/60} min(s)")
    return lazo

def pointcloud_to_dtmtiff(laz,tif,lazo,res=10,outfn="mean",method="smrf"):
    qi,pipeline = pdal_quick_info(laz)
    unique_classes = pdal_pc_classes(pipeline)
    # Check if 2 is in unique_classes
    if 2 in unique_classes:
        print('YES:::PointCloud ALREADY classified...')
        generate_dtm(laz, tif, res=res,outfn=outfn)
    else:
        print('NO:::PointCloud NOT YET classified...')
        lazo = clf_pointcloud(laz, lazo, method=method)
        generate_dtm(lazo, tif, res=res,outfn=outfn)

In [4]:
for i, location in enumerate(transect_names):
    if i > 1: break
    
    loc_laz_files = glob(f"{eba_laz_dir}/{location}/*.laz")
    loc_tif_dir = f"{eba_tif_dir}/{location}"
    loc_lazo_dir = f"{eba_lazclf_dir}/{location}"
    os.makedirs(loc_tif_dir, exist_ok=True)
    os.makedirs(loc_lazo_dir, exist_ok=True)
    print(f"{i}:{location} :: {len(loc_laz_files)} laz")
    for laz in loc_laz_files:
        btif = os.path.basename(laz).replace('.laz', '.tif')
        tif = f"{loc_tif_dir}/{btif}"
        lazo = f"{loc_lazo_dir}/{os.path.basename(laz)}"
        pointcloud_to_dtmtiff(laz,tif,lazo,res=10,outfn="mean",method="smrf")

0:transectos_tocantins :: 1 laz
Unique classifications: [0 2 4]
YES:::PointCloud ALREADY classified...
Generating DTM: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_mean.tif
DTM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_DTM10_mean.tif
run.time = 0.37328819793328877 min(s)
1:transectos_para_p5 :: 9 laz
Unique classifications: [0 2 4]
YES:::PointCloud ALREADY classified...
Generating DTM: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_para_p5/NP_T-1016_001.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_para_p5/NP_T-1016_001_DTM10_mean.tif
DTM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_para_p5/NP_T-1016_001_DTM10_mean.tif
run.time = 0.13806041015001635 min(s)
Unique classifications: [0 2 4]
YES:::PointCloud ALREADY classified...

In [10]:
def pointcloud2dtm(laz,tif,lazo,res=10,outfn="mean",method="smrf"):
    qi,pipeline = pdal_quick_info(laz)
    unique_classes = pdal_pc_classes(pipeline)
    # Check if 2 is in unique_classes
    if 2 in unique_classes:
        print('YES:::PointCloud ALREADY classified...')
        generate_dtm(laz, tif, res=res,outfn=outfn)
    else:
        print('NO:::PointCloud NOT YET classified...')
        lazo = clf_pointcloud(laz, lazo, method=method)
        generate_dtm(lazo, tif, res=res,outfn=outfn)

In [20]:


qi,pipeline = pdal_quick_info(laz)
unique_classes = pdal_pc_classes(pipeline)
# Check if 2 is in unique_classes
if 2 in unique_classes:
    print('Yes')
    generate_dtm(laz, tif, res=10,outfn="mean")
else:
    print('N')

Unique classifications: [0 2 4]
Yes
